<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ItsShi/MonoDepth.git

!pip -q install tensorboardX==1.4
!pip -q install torchvision==0.12.0
!pip -q install transformers

Cloning into 'MonoDepth'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 166 (delta 66), reused 83 (delta 27), pack-reused 4
Receiving objects: 100% (166/166), 10.44 MiB | 30.44 MiB/s, done.
Resolving deltas: 100% (66/66), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.11

In [ ]:
import gdown

video_url = 'https://drive.google.com/uc?id=1c_ewx6wts7pJTb3XVWVXEZLbFXDObpdP'
gdown.download(video_url,'SCARED_video.zip',quiet=True) 

!unzip -q /content/SCARED_video.zip

In [2]:
# depth gt
!pwd
import gdown
import shutil

scene_url = 'https://drive.google.com/uc?id=1xZwB0qFWbU7EjjI1g-Y0pqAs1_nWwU6Z'
gdown.download(scene_url, 'scene.tar.gz', quiet=True)

shutil.unpack_archive('scene.tar.gz', 'd3k1_tiffs')

/content


In [4]:
# vid2frame
!pip install ffmpeg
!mkdir /content/data
!ffmpeg -i /content/d3k1_rgb.mp4 %6d.jpg  
!mv *.jpg /content/data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=136c80e9897a007a02462acb7df75951324f09704edf54693bfb4922b0632dd0
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontc

In [3]:
# test_files
!touch train_files.txt
!touch val_files.txt
!touch test_files.txt

for opt in ["train", "val", "test"]:
    f = open("/content/MonoDepth/splits/endovis/{}_files.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if folder == "dataset3/keyframe4" and int(frame)<328:
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

# move and replace the original train_files.txt file
%pwd
!cp /content/MonoDepth/splits/endovis/test_files.txt /content/test_files_original.txt
!cp -f /content/val_files.txt /content/MonoDepth/splits/endovis/val_files.txt
!cp -f /content/train_files.txt /content/MonoDepth/splits/endovis/train_files.txt
!cp -f /content/test_files.txt /content/MonoDepth/splits/endovis/test_files.txt

0
0
34


In [7]:
%cd MonoDepth
!python export_gt_depth.py --data_path /content/d3k1_tiffs --split endovis
%cd ..

/content/MonoDepth
Exporting ground truth depths for endovis
1
dataset3/keyframe4
2
dataset3/keyframe4
3
dataset3/keyframe4
4
dataset3/keyframe4
5
dataset3/keyframe4
6
dataset3/keyframe4
7
dataset3/keyframe4
8
dataset3/keyframe4
9
dataset3/keyframe4
10
dataset3/keyframe4
11
dataset3/keyframe4
12
dataset3/keyframe4
13
dataset3/keyframe4
14
dataset3/keyframe4
15
dataset3/keyframe4
16
dataset3/keyframe4
17
dataset3/keyframe4
18
dataset3/keyframe4
19
dataset3/keyframe4
20
dataset3/keyframe4
21
dataset3/keyframe4
22
dataset3/keyframe4
23
dataset3/keyframe4
24
dataset3/keyframe4
25
dataset3/keyframe4
26
dataset3/keyframe4
27
dataset3/keyframe4
28
dataset3/keyframe4
29
dataset3/keyframe4
30
dataset3/keyframe4
31
dataset3/keyframe4
32
dataset3/keyframe4
33
dataset3/keyframe4
34
dataset3/keyframe4
Saving to endovis
/content


In [6]:
%cd MonoDepth
!python evaluate_depth.py --dpt --eval_split endovis --data_path /content/data --eval_mono  --width 320 --height 256 
# --load_weights_folder 
%cd ..

/content/MonoDepth
2023-06-09 15:27:17.419392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initi